# UE: Numerical Methods for Partial Differential Equations

## Exercise 2.4

Additional notes on the calculation of the exact values can be found in my submission "Ex2.4.pdf".

Submission date: 17.3.2020

Author: Peter Holzner, 1426733

In [ ]:
# First import the necessary modules
import netgen.gui
from ngsolve import *
import netgen.geom2d as geom2dim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Analytical (exact) values
# Since -delta(u)=x, we assume that u is polynomial of third order
# by fiddling around with the boundary conditions, we "guessed" that...
# u_exact = x*(1-x²)/6 => simulation should be exact for k=3
Wr_exact = -1/3
Wl_exact = 1/6
Wd_exact = Wr_exact + Wl_exact
#---------------------------------------------------------#
ks = [1, 2, 3]
hs = [0.5**i for i in range(1,8)]
fig, ax = plt.subplots(3, figsize= [8,6])
fig.figsize = [6.4*2, 4.8*2]
for i, k in enumerate(ks):
    print('#'*30)
    print('#'+' '*12+'k={}'.format(k)+' '*13+'#')
    print('#'*30)
    # Gather all results/errors for one "k" in the containers
    Wl_errors = []
    Wr_errors = []
    Wd_errors = []
    #
    for mesh_size in hs:
        ####### Setup #######
        mesh = Mesh(geom2dim.unit_square.GenerateMesh(maxh=mesh_size))
        fes = H1(mesh, order=k, dirichlet='right|left')

        # Set trial and test function
        u = fes.TrialFunction()
        v = fes.TestFunction()

        # Set right hand side
        rhs = LinearForm(fes)
        rhs += x*v*dx

        # The bilinear form
        A = BilinearForm(fes, symmetric=True)
        A += grad(u)*grad(v)*dx

        # Now assemble the system of equations
        with TaskManager():
            rhs.Assemble()
            A.Assemble()

        ###### Calculations #######
        # Calculate the solution field (function)
        gf = GridFunction(fes)
        gf.vec.data = A.mat.Inverse(fes.FreeDofs(), inverse='sparsecholesky') * rhs.vec

        # Calculate temperature flux through left and right boundaries
        dudx = GridFunction(fes)
        dudx.Set(grad(gf)[0])

        # Interpolation and integration by "hand"
        pts = [(0,i) for i in np.arange(0, 1+mesh_size, mesh_size)]
        vals = [grad(gf)(mesh(x,y))[0] for x, y in pts]
        #print("vals:", vals)
        # Integration using trapezoidal rule for boundary "left"
        sum = 0
        for val1, val2 in zip(vals[:-1], vals[1:]):
            sum += (val1 + val2)/2*mesh_size

        # Option 1:
        # Wl = Integrate(dudx, mesh, BND, region_wise=True)
        # BND... evaluate integral on boundary

        # Option 2:
        Wl = Integrate(dudx, mesh, definedon=mesh.Boundaries('left'))
        Wr = Integrate(dudx, mesh, definedon=mesh.Boundaries('right'))
        Wd = Wl + Wr
        # Append results to container
        Wl_errors.append(Wl)
        Wr_errors.append(Wr)
        Wd_errors.append(Wd)

        # Check if control integration by trap is equal to Wl
        if np.abs(Wl - sum) <=2e-15:
            print("Integration check: <success>")
        else:
            print("Integration check: <failure>")

        ###### Output to console ########
        print("mesh size:", mesh_size)
        print("Trapezoidal:", sum, " ... should be equal to next line (Wl=)")
        print("Wl=", Wl, "...within 1 eps of exact? <", np.allclose(Wl, Wl_exact, atol=Wl_exact*2e-16), ">")
        print("Wr=", Wr, "...within 1 eps of exact? <", np.allclose(Wr, Wr_exact, atol=Wr_exact*2e-16), ">")
        print("Total flux through dirichlet boundaries:")
        print("Wd= ", Wd, "...within 1 eps of exact? <", np.allclose(Wd, Wd_exact, atol=Wd_exact*2e-16), ">")
        print('-'*30)

    ####### ALL MESH SIZES DONE ##########
    # Calculate errors after all mesh sizes are done for this k
    Wl_errors = np.abs(np.array(Wl_errors) - Wl_exact)
    Wr_errors = np.abs(np.array(Wr_errors) - Wr_exact)
    Wd_errors = np.abs(np.array(Wd_errors) - Wd_exact)
    # Plot in subplot
    ax[i].loglog(hs, Wl_errors, '-xr', label='Wl')
    ax[i].loglog(hs, Wr_errors, '-pb', label='Wr')
    ax[i].loglog(hs, Wd_errors, '-xg', label='Wd')
    ax[i].set_title("k={}".format(k))
    ax[i].set_xlabel("h... mesh size")
    ax[i].grid(True)
    ax[i].legend()

# Print exact values to console
print('#'*30)
print("Analytical (exact) values:")
print("Wl_exact=", Wl_exact)
print("Wr_exact=", Wr_exact)
print("Wd_exact=", Wd_exact)


fig.suptitle("Error of temperature flux through the Dirichlet boundaries")
fig.show()

In [ ]:
# Save figure as png
fig.savefig("./Ex2.4_flux-error.png")

In [ ]:
# Draw the latest solution and mesh
Draw(gf, mesh, 'grid_function')
Draw(grad(gf), mesh, 'gradient_gf')